In [357]:
import pandas as pd
import numpy as np

## 1. Wstępna analiza

In [358]:
# Zaimportuj dane

ds_salaries = pd.read_csv('ds_salaries.csv')
ds_salaries

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [359]:
# Pośród 3755 rekordów nie ma żadnych wartości null
print(f"Krztałt zbioru danych: {ds_salaries.shape}\n")

ds_salaries.info()

print(f'\n Zbiór danych nie ma żadnyh wartości null')

Krztałt zbioru danych: (3755, 11)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB

 Zbiór danych nie ma żadnyh wartości null


In [360]:
print('Kolumna remote_ratio jest nietypowa ponieważ ma typ zmiennych int, ale wartości są zdecydowanie kategoryczne. Zamieniam je na bardziej czytelne')

ds_salaries['remote_ratio'].unique()
ds_salaries['remote_ratio'] = ds_salaries['remote_ratio'].replace({100: 'Remote', 0:'On-site', 50:'Hybrid'})
ds_salaries['remote_ratio'].describe()

Kolumna remote_ratio jest nietypowa ponieważ ma typ zmiennych int, ale wartości są zdecydowanie kategoryczne. Zamieniam je na bardziej czytelne


count        3755
unique          3
top       On-site
freq         1923
Name: remote_ratio, dtype: object

In [361]:
object_columns = ds_salaries.select_dtypes(include=['object']).columns

print(f'W zbiorze danych jest {len(object_columns)} kolumn o typie innej niż liczbowy')

W zbiorze danych jest 8 kolumn o typie innej niż liczbowy


In [362]:
for col in object_columns:
    print(f'Kolumna {col} ma {len(ds_salaries[col].unique())} różnych wartości')

Kolumna experience_level ma 4 różnych wartości
Kolumna employment_type ma 4 różnych wartości
Kolumna job_title ma 93 różnych wartości
Kolumna salary_currency ma 20 różnych wartości
Kolumna employee_residence ma 78 różnych wartości
Kolumna remote_ratio ma 3 różnych wartości
Kolumna company_location ma 72 różnych wartości
Kolumna company_size ma 3 różnych wartości


## 2. Dostosowanie kolumn nieliczbowych

In [363]:
from sklearn.preprocessing import LabelEncoder

In [364]:
data = ds_salaries.copy()

data_obj = data[object_columns]

labelEncoder = LabelEncoder()

for col in object_columns:
    data[col] = labelEncoder.fit_transform(data[col])

print("Zdecydowałem się na użycie LabelEncodera, ponieważ przy zbiór danych ma kilka kolumn nienumerycznych, które mają sporo różnych wartości.")
print("Niestety nie mają one teraz wartości 0-1, dlatego będzie konieczne przesklawoanie tych wartości")

Zdecydowałem się na użycie LabelEncodera, ponieważ przy zbiór danych ma kilka kolumn nienumerycznych, które mają sporo różnych wartości.
Niestety nie mają one teraz wartości 0-1, dlatego będzie konieczne przesklawoanie tych wartości


## 3. Podział danych i ich skalowanie

In [365]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline

In [366]:
y = data['salary_in_usd']
X = data.drop('salary_in_usd', axis=1)  #To tworzy kopię data i dopiero wtedy usuwa kolumnę, więc oryginalna data jest nietknięta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [367]:
preprocessing_pipeline = make_pipeline(
    MinMaxScaler(),
)

print("Ten potok pozwala na przeskalowanie wartości do zakresu <0; 1> przy pomocy metody MinMaxScaler")

Ten potok pozwala na przeskalowanie wartości do zakresu <0; 1> przy pomocy metody MinMaxScaler


## 4. DecisionTreeClassifier

In [368]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
import numpy as np

In [369]:
dtc_pipeline = Pipeline([
    ('preprocessing', preprocessing_pipeline),
    ('classification', DecisionTreeClassifier())
])

In [370]:
dtc_params = {
    'classification__criterion':['gini','entropy'],
    'classification__max_depth':np.arange(1, 21).tolist()[0::2],
    'classification__min_samples_split':np.arange(2,11).tolist()[0::2],
    'classification__max_leaf_nodes':np.arange(3,501).tolist()[0::2]
}

In [371]:
r1 = RandomizedSearchCV(dtc_pipeline, dtc_params, cv=5, n_iter=100, random_state=42)

In [372]:
r1.fit(X_train, y_train)

c:\Users\Stasiek\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessing',
                                              Pipeline(steps=[('minmaxscaler',
                                                               MinMaxScaler())])),
                                             ('classification',
                                              DecisionTreeClassifier())]),
                   n_iter=100,
                   param_distributions={'classification__criterion': ['gini',
                                                                      'entropy'],
                                        'classification__max_depth': [1, 3, 5,
                                                                      7, 9, 11,
                                                                      13, 15,
                                                                      17, 19],
                                        'classification__max_leaf_nodes': [3, 5,
                                                                           7, 9,
                                                                           11,
                                                                           13,
                                                                           15,
                                                                           17,
                                                                           19,
                                                                           21,
                                                                           23,
                                                                           25,
                                                                           27,
                                                                           29,
                                                                           31,
                                                                           33,
                                                                           35,
                                                                           37,
                                                                           39,
                                                                           41,
                                                                           43,
                                                                           45,
                                                                           47,
                                                                           49,
                                                                           51,
                                                                           53,
                                                                           55,
                                                                           57,
                                                                           59,
                                                                           61, ...],
                                        'classification__min_samples_split': [2,
                                                                              4,
                                                                              6,
                                                                              8,
                                                                              10]},
                   random_state=42)

In [373]:
r1.best_params_

{'classification__min_samples_split': 4,
 'classification__max_leaf_nodes': 425,
 'classification__max_depth': 17,
 'classification__criterion': 'gini'}

In [374]:
r1.score(X_test, y_test)

0.711051930758988

In [380]:
y_pred = r1.predict(X_test)

print(f'Accuracy score: {accuracy_score(y_true=y_test, y_pred=y_pred)}')
print(f'Precyzja: {precision_score(y_true=y_test, y_pred=y_pred, average="weighted")}')
print(f'Czułość: {recall_score(y_true=y_test, y_pred=y_pred, average="weighted")}')

Accuracy score: 0.711051930758988
Precyzja: 0.683952532154929
Czułość: 0.711051930758988


c:\Users\Stasiek\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Stasiek\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 5. KNeighborsClassifier